<h2> Import the required packages</h2>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelt

<h3> Read the Label data and predictor features </h3>

In [2]:
# to_yymmdd = lambda dt: datetime.strptime(dt, "%d%m%d")
to_mmddyyyyHHMM = lambda dt: datetime.strptime(dt, "%m/%d/%Y %H:%M")
df_power=pd.read_csv('power_actual.csv',index_col=0,converters={'datetime':to_mmddyyyyHHMM})
df_wether=pd.read_csv('weather_actuals.csv',index_col=0,converters={'datetime_utc':to_mmddyyyyHHMM,'datetime_local':to_mmddyyyyHHMM,'sunrise':to_mmddyyyyHHMM,'sunset':to_mmddyyyyHHMM,'updated_at':to_mmddyyyyHHMM})

<h4> Looking at the unique values in columns ghi and gti, we can say that these columns are either irrelevant for the data provided for these columns are incorrect.<br>
Hence we can drop these columns</h4>

In [3]:
print(df_power.ghi.unique())
print(df_power.gti.unique())
print(df_power.power.unique())

[ 0. nan]
[ 0. nan]
[0.00000e+00 8.00000e-02 3.40000e-01 ... 5.60224e+03 3.92693e+03
 2.61160e+03]


In [4]:
df_power.drop(['ghi','gti'],axis=1,inplace=True)

<h3>Check for NAN values</h3>

In [5]:
df_power.isna().sum()

datetime    0
power       0
dtype: int64

<h3>Check for the duration within a day when solar power is generated</h3>
<h4>Looking at the given data we can see that Power is generated on between day time i.e between 6am to 6pm</h4>

In [6]:
for hh in range(5,11):
    df_power2=df_power[(df_power.datetime.dt.hour>=hh) & (df_power.datetime.dt.hour<=hh+12)]
    print('Number of records, power generated after ',hh,' AM and before ',hh,' PM:',df_power2[df_power2.power!=0].shape[0])
    print('Number of records, no power generated after ',hh,' AM and before ',hh,' PM:',df_power2[df_power2.power==0].shape[0])
del(df_power2)    
df_power2=df_power[(df_power.datetime.dt.hour<6) | (df_power.datetime.dt.hour>18)]
print('Total Number of records:',df_power2.shape[0])
print('Number of records, power generated before 6 AM and after 6 PM:',df_power2[df_power2.power!=0].shape[0])
print('Number of records, no power generated before 6 AM and after 6 PM:',df_power2[df_power2.power==0].shape[0])
del(df_power2)

Number of records, power generated after  5  AM and before  5  PM: 27998
Number of records, no power generated after  5  AM and before  5  PM: 9962
Number of records, power generated after  6  AM and before  6  PM: 27998
Number of records, no power generated after  6  AM and before  6  PM: 9962
Number of records, power generated after  7  AM and before  7  PM: 26900
Number of records, no power generated after  7  AM and before  7  PM: 11060
Number of records, power generated after  8  AM and before  8  PM: 24624
Number of records, no power generated after  8  AM and before  8  PM: 13336
Number of records, power generated after  9  AM and before  9  PM: 22136
Number of records, no power generated after  9  AM and before  9  PM: 15824
Number of records, power generated after  10  AM and before  10  PM: 19619
Number of records, no power generated after  10  AM and before  10  PM: 18341
Total Number of records: 32120
Number of records, power generated before 6 AM and after 6 PM: 0
Number o

<h3>Selected the time frame between 6AM to 6PM as out of this window there is no power generated.</h3>

In [7]:
df_power=df_power[(df_power.datetime.dt.hour>=6) & (df_power.datetime.dt.hour<=18)]
print('Number of records, power generated after 6 AM and before 6 PM:',df_power[df_power.power!=0].shape[0])
print('Number of records, no power generated after 6 AM and before 6 PM:',df_power[df_power.power==0].shape[0])
df_power.sort_values('datetime',ascending=True,inplace=True)
df_wether2=df_wether[(df_wether.datetime_local.dt.hour>=6) & (df_wether.datetime_local.dt.hour<=18)]

Number of records, power generated after 6 AM and before 6 PM: 27998
Number of records, no power generated after 6 AM and before 6 PM: 9962


<h4>Check for the NAN values and treat them</h4>

In [8]:
df_wether2.isna().sum()[df_wether2.isna().sum()>0]

wind_chill             5349
heat_index             5349
qpf                    5349
snow                   5349
pop                    5349
fctcode                5349
precip_accumulation    5349
precip_type            3723
dtype: int64

In [9]:
print(df_wether2.wind_chill.unique())
print(df_wether2.heat_index.unique())
print(df_wether2.qpf.unique())
print(df_wether2.snow.unique())
print(df_wether2['pop'].unique())
print(df_wether2.fctcode.unique())
print(df_wether2.precip_accumulation.unique())
print(df_wether2.precip_type.unique())

[-9999.    nan]
[-9999.    nan]
[-9999.    nan]
[-9999.    nan]
[-9999.    nan]
[-9999.    nan]
[-9999.    nan]
['rain' '-9999' nan]


<h4> Looking at the unique values in columns except for Precip_type column, we can say that these columns are either irrelevant or the data provided for these columns are incorrect.<br>
Hence we can drop these columns and treat Precip_type column</h4>
<h4>At the same time we will sort the data on date time</h4>

In [10]:
df_wether2.drop(['wind_chill','heat_index','qpf','snow','pop','fctcode','precip_accumulation'],axis=1,inplace=True)
df_wether2.fillna('-9999',inplace=True)
df_wether2.sort_values('datetime_local',inplace=True)

C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs
C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate f

<h3> Check for data type of all the features</h3>

In [11]:
df_wether2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377 entries, 6 to 13613
Data columns (total 23 columns):
plant_id                7377 non-null int64
datetime_utc            7377 non-null datetime64[ns]
datetime_local          7377 non-null datetime64[ns]
cloud_cover             7377 non-null float64
apparent_temperature    7377 non-null float64
temperature             7377 non-null int64
humidity                7377 non-null int64
dew_point               7377 non-null float64
wind_bearing            7377 non-null int64
wind_speed              7377 non-null float64
wind_gust               7377 non-null float64
pressure                7377 non-null float64
uv_index                7377 non-null int64
ozone                   7377 non-null float64
precip_intensity        7377 non-null float64
precip_probability      7377 non-null float64
precip_type             7377 non-null object
visibility              7377 non-null float64
sunrise                 7377 non-null datetime64[ns]
sunset  

<h3> Function to create a ordinal feature for hour of the day</h3>

In [12]:
def get_pahar(hr):
    """ We will give more weight to afternoon hours as during this time solar heat and radiation is more.
    Resulting in more generation of solar power"""
    if (hr>=6 and hr<9): return(4)    #Sunrise time
    elif hr>=9 and hr<12: return(3)   #Morning Time
    elif hr>=12 and hr<15: return(1)  #Afternoon
    elif hr>=15 and hr<=18: return(2) #Evening

<h3>Check for the categorical variables in whethers data</h3>

In [13]:
print('Summary:\n',df_wether2.summary.unique())
print('Icon:\n',df_wether2.icon.unique())
print('precip_type:\n',df_wether2.precip_type.unique())

Summary:
 ['Clear' 'Foggy' 'Partly Cloudy' 'Mostly Cloudy' 'Breezy and Foggy'
 'Overcast' 'Humid and Partly Cloudy' 'Humid and Mostly Cloudy'
 'Humid and Overcast' 'Humid and Foggy' 'Humid' 'Possible Light Rain'
 'Rain' 'Light Rain' 'Light Rain and Humid'
 'Possible Light Rain and Humid' 'Possible Drizzle and Humid'
 'Rain and Humid']
Icon:
 ['clear-night' 'fog' 'clear-day' 'partly-cloudy-day' 'partly-cloudy-night'
 'wind' 'cloudy' 'rain']
precip_type:
 ['rain' '-9999']


<h4><li>One hot encode the categorical variables</li>
    <li>Generate a new feature to get the week of the year</li>
    <li>Generate a new feature to specify the time of day as pahar</li>
    <li>Drop not required features from the dataset</li>
</h4>

In [14]:
df_wether2=pd.concat([df_wether2,pd.get_dummies(df_wether2.precip_type,prefix='precip_type')],axis=1)
df_wether2=pd.concat([df_wether2,pd.get_dummies(df_wether2.summary,prefix='summary')],axis=1)
df_wether2=pd.concat([df_wether2,pd.get_dummies(df_wether2.icon,prefix='icon')],axis=1)
df_wether2['week_of_year']=df_wether2.datetime_local.dt.weekofyear
df_wether2['pahar']=df_wether2.datetime_local.dt.hour.apply(lambda hr: get_pahar(hr))
df_wether2.drop(['icon','plant_id','summary','sunrise','sunset','updated_at','precip_type','datetime_utc'],axis=1,inplace=True)

<h3> For using time series algorithms we need to have a continous time series data,<br>
    Check if we have Missing Days in the data</h3>

In [15]:
print('Counts for Whether Data')
print('Required no. of Days: ',(df_wether2.datetime_local.max()-df_wether2.datetime_local.min()).days+1,\
        '\nExisting no. of Days: ',len(df_wether2.datetime_local.dt.date.unique()),\
        '\nMissing no. Of Days: ',(df_wether2.datetime_local.max()-df_wether2.datetime_local.min()).days-len(df_wether2.datetime_local.dt.date.unique())+1)

Counts for Whether Data
Required no. of Days:  730 
Existing no. of Days:  568 
Missing no. Of Days:  162


In [16]:
print('Counts for Power Data')
print('Required no. of Days: ',(df_power.datetime.max()-df_power.datetime.min()).days+1,\
        '\nExisting no. of Days: ',len(df_power.datetime.dt.date.unique()),\
        '\nMissing no. Of Days: ',(df_power.datetime.max()-df_power.datetime.min()).days-len(df_power.datetime.dt.date.unique())+1)

Counts for Power Data
Required no. of Days:  730 
Existing no. of Days:  730 
Missing no. Of Days:  0


<h4> Looking at list of missing days we can clearly see that data is missing for dates between 28th of Feb 2018 till 15th Aug 2018.</h4>
<p>Imputing data for these many days consecutively will degrade the performance of models.
Hence we can ignore this missing dates for now.</p>

In [17]:
d=df_wether2.sort_values('datetime_local').datetime_local.to_list()
print('DataDate count:',len(d))
date_set = set(d[0] + timedelta(x) for x in range((d[-1] - d[0]).days))
print('Actual Date count:',len(date_set))
missing = sorted(date_set - set(d))
print('Missing Date count:',len(missing))
print([datetime.strftime(dt,"%d%B%Y") for dt in missing])

DataDate count: 7377
Actual Date count: 729
Missing Date count: 163
['28February2018', '01March2018', '02March2018', '03March2018', '04March2018', '05March2018', '06March2018', '13March2018', '14March2018', '15March2018', '16March2018', '17March2018', '18March2018', '19March2018', '20March2018', '21March2018', '22March2018', '23March2018', '24March2018', '25March2018', '26March2018', '27March2018', '28March2018', '29March2018', '30March2018', '31March2018', '01April2018', '02April2018', '03April2018', '04April2018', '05April2018', '06April2018', '07April2018', '08April2018', '09April2018', '10April2018', '11April2018', '12April2018', '13April2018', '14April2018', '15April2018', '16April2018', '17April2018', '18April2018', '19April2018', '20April2018', '21April2018', '22April2018', '23April2018', '24April2018', '25April2018', '26April2018', '27April2018', '28April2018', '29April2018', '30April2018', '01May2018', '02May2018', '03May2018', '04May2018', '05May2018', '06May2018', '07May2018

<h3> We will join the Power data and Wether data generating new preprocessed data having all the features for each 15min interval time</h3>

In [18]:
df_wether2['Date']=df_wether2.datetime_local.dt.strftime('%Y-%m-%d %HH')
df_power['Date']=df_power.datetime.dt.strftime('%Y-%m-%d %HH')
df_15min_int_data=df_power.join(df_wether2.set_index('Date'),on=['Date'],how='inner').drop(['Date','datetime_local'],axis=1)

<h3>Check if combined data is having any NAN Values</h3>

In [19]:
df_15min_int_data.isna().sum()

datetime                                 0
power                                    0
cloud_cover                              0
apparent_temperature                     0
temperature                              0
humidity                                 0
dew_point                                0
wind_bearing                             0
wind_speed                               0
wind_gust                                0
pressure                                 0
uv_index                                 0
ozone                                    0
precip_intensity                         0
precip_probability                       0
visibility                               0
precip_type_-9999                        0
precip_type_rain                         0
summary_Breezy and Foggy                 0
summary_Clear                            0
summary_Foggy                            0
summary_Humid                            0
summary_Humid and Foggy                  0
summary_Hum

<h3>Save the data frame to a file</h3>

In [20]:
df_15min_int_data.to_csv('climate_15min_int_data.csv')